In [1]:
import pandas as pd
import random

initial_data = {
    "ID": ["A01", "A02", "A03", "A04", "A05", "A06", "A07", "A08"],
    "Latitude": [-6.292, -6.295, -6.298, -6.293, -6.291, -6.296, -6.294, -6.297],
    "Longitude": [106.784, 106.781, 106.789, 106.786, 106.783, 106.782, 106.788, 106.785],
    "Time": ["07:45", "17:30", "21:10", "08:15", "14:20", "16:40", "09:10", "22:00"]
}
initial_df = pd.DataFrame(initial_data)

def time_interval(time):
    hour = int(time.split(":")[0])
    if 6 <= hour <= 9:
        return "Pagi"
    elif 10 <= hour <= 14:
        return "Siang"
    elif 15 <= hour <= 18:
        return "Sore"
    else:
        return "Malam"

initial_df["Interval"] = initial_df["Time"].apply(time_interval)

target_counts = {
    "Pagi": 28,
    "Sore": 25,
    "Malam": 13,
    "Siang": 14
}

final_df_parts = []

last_id_num = initial_df['ID'].str.replace('A', '', regex=False).astype(int).max()


for interval, target_count in target_counts.items():
    current_interval_df = initial_df[initial_df['Interval'] == interval].copy()
    current_count = len(current_interval_df)

    if current_count < target_count:

        needed = target_count - current_count
        new_rows_for_interval = []
        representative_row = None
        if not current_interval_df.empty:

            representative_row = current_interval_df.sample(1, random_state=42).iloc[0]

        for _ in range(needed):
            last_id_num += 1
            new_id = f"A{last_id_num:02d}"

            if representative_row is not None:

                new_lat = representative_row['Latitude'] + random.uniform(-0.0005, 0.0005)
                new_lon = representative_row['Longitude'] + random.uniform(-0.0005, 0.0005)


                if interval == "Pagi": new_time = f"{random.randint(6,9):02d}:{random.randint(0,59):02d}"
                elif interval == "Siang": new_time = f"{random.randint(10,14):02d}:{random.randint(0,59):02d}"
                elif interval == "Sore": new_time = f"{random.randint(15,18):02d}:{random.randint(0,59):02d}"
                else: new_time = f"{random.choice([random.randint(0,5), random.randint(19,23)]):02d}:{random.randint(0,59):02d}"

                new_rows_for_interval.append({
                    "ID": new_id,
                    "Latitude": new_lat,
                    "Longitude": new_lon,
                    "Time": new_time,
                    "Interval": interval
                })
            else:

                default_lat = -6.295
                default_lon = 106.785
                default_time = "12:00"
                if interval == "Pagi": default_time = "08:00"
                elif interval == "Siang": default_time = "12:00"
                elif interval == "Sore": default_time = "17:00"
                else: default_time = "22:00"

                new_rows_for_interval.append({
                    "ID": new_id,
                    "Latitude": default_lat,
                    "Longitude": default_lon,
                    "Time": default_time,
                    "Interval": interval
                })


        if new_rows_for_interval:
            current_interval_df = pd.concat([current_interval_df, pd.DataFrame(new_rows_for_interval)], ignore_index=True)

    elif current_count > target_count:

        rows_to_remove = current_count - target_count

        current_interval_df = current_interval_df.sample(n=target_count, random_state=42)


    final_df_parts.append(current_interval_df)


df = pd.concat(final_df_parts, ignore_index=True)


df.sort_values(by='ID', inplace=True, kind='mergesort')


count_interval = df["Interval"].value_counts()

display(df)
display(count_interval)

,ID,Latitude,Longitude,Time,Interval
0,A01,-6.292000,106.784000,07:45,Pagi
28,A02,-6.295000,106.781000,17:30,Sore
53,A03,-6.298000,106.789000,21:10,Malam
1,A04,-6.293000,106.786000,08:15,Pagi
66,A05,-6.291000,106.783000,14:20,Siang
...,...,...,...,...,...
75,A76,-6.290530,106.782707,14:14,Siang
76,A77,-6.291358,106.782852,14:56,Siang
77,A78,-6.291141,106.783277,13:18,Siang
78,A79,-6.291214,106.783417,10:43,Siang


,count
Interval,
Pagi,28
Sore,25
Siang,14
Malam,13


In [ ]:
import folium

# Titik tengah peta
m = folium.Map(location=[-6.294, 106.785], zoom_start=14)

# Warna per interval
color_map = {
    "Pagi": "red",
    "Siang": "orange",
    "Sore": "blue",
    "Malam": "purple"
}

# Plot titik
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=6,
        color=color_map[row["Interval"]],
        fill=True,
        fill_opacity=0.7,
        popup=f"{row['ID']} - {row['Interval']}"
    ).add_to(m)

m
